In [1]:
import pandas as pd
from datetime import time
from database.db_utils import get_connection
pd.set_option('display.max_columns', None)

## since, the smallest unit for which we have to measure the availability is an hour.
## so, divide max(timestamp) - min(timestamp) into batches of 1 hour for each day.
## look for available data in each of the batches.
## if the store was active we mark the whole hour as available.
## if the store was not active or the data is not available then we mark the whole hour as not available.

In [2]:
# connect to the database
engine = get_connection()

# load the tables into pd.Dataframes
store_status_df: pd.DataFrame = pd.read_sql_table('store_status', con=engine)
time_zone_df: pd.DataFrame = pd.read_sql_table('time_zone', con=engine)
menu_hours_df: pd.DataFrame = pd.read_sql_table('menu_hours', con=engine)
reports_df: pd.DataFrame = pd.read_sql_table('reports', con=engine)

In [7]:
merged_df = pd.merge(store_status_df, menu_hours_df, on='store_id', how='left')
merged_df = pd.merge(merged_df, time_zone_df, on='store_id', how='left')

merged_df['timestamp_utc'] = pd.to_datetime(merged_df['timestamp_utc'])
merged_df['timestamp_utc'] = merged_df['timestamp_utc'].dt.tz_localize('UTC')

merged_df['timezone_str'] = merged_df['timezone_str'].fillna('America/Chicago')
merged_df['timestamp_local'] = merged_df\
    .apply(lambda row: row['timestamp_utc'].tz_convert(row['timezone_str']), axis=1)
grouped = merged_df.groupby(by=['store_id'])\
    .agg(min_value=('timestamp_local', 'min'), max_value=('timestamp_local', 'max'))

print(grouped)

                                     min_value                  max_value
store_id                                                                 
257406274356679      2023-01-18 08:31:37-05:00  2023-01-25 13:03:20-05:00
309081616338296      2023-01-18 15:11:02-05:00  2023-01-25 13:09:53-05:00
2570905277901393     2023-01-17 19:00:35-05:00  2023-01-25 13:05:39-05:00
2843799253836859     2023-01-18 14:52:58-05:00  2023-01-25 13:00:26-05:00
3225796639170927     2023-01-18 08:51:00-05:00  2023-01-25 13:03:12-05:00
...                                        ...                        ...
9218470229355806679  2023-01-18 13:48:09-06:00  2023-01-25 12:02:31-06:00
9219648204163704298  2023-01-17 19:11:15-05:00  2023-01-25 13:04:43-05:00
9220622754425525111  2023-01-18 14:42:50-05:00  2023-01-25 13:08:03-05:00
9221835456246842967  2023-01-18 14:38:04-05:00  2023-01-25 13:05:19-05:00
9222367421889482050  2023-01-18 14:43:21-05:00  2023-01-25 13:12:13-05:00

[14092 rows x 2 columns]
            